# Evalution and Analysis

## Imports:

In [ ]:
from pathlib import Path
from Helper.evaluation_helpers import get_predictions, calc_metrics, error_plot,plot_metric
import warnings
from matplotlib import pyplot as plt

### Supress warnings

In [ ]:
warnings.filterwarnings("ignore") 

In [ ]:
import numpy as np
from pathlib import Path

predsPath = Path('../Predictions/')
for i in list(predsPath.glob('*.npy')):
    data = np.load(i)
    print(f" {i.name} has {len(data)} values")  # Show length of each prediction file


## Supress warnings

In [ ]:
import warnings
warnings.filterwarnings("ignore") 

In [ ]:
predsPath= Path('../Predictions/')
predsDf=get_predictions(predsPath)
display(predsDf)

# Calculate Metrics:

In [ ]:
# metric DataFrames are sorted by the main metric before being displayed
main_metric = 'RMSE'

# Horizon = 1:

In [ ]:
predsPath_1= Path('../Predictions/Horizon1')
predsDf_1=get_predictions(predsPath_1)

metrics_1= calc_metrics(predsDf_1)
display(metrics_1.sort_values(main_metric, axis=0))

error_plot(predsDf_1,model='ARIMA1990',absolute=True, title= 'ARIMA1990 Horizon 1 Error plot:')

error_plot(predsDf_1,model=['ARIMA1990','ARIMAX1990', 'SARIMA1990','SARIMAX1990'],absolute=True, title='Horizon 1 Error plot: ')
error_plot(predsDf_1,model='all',absolute=False, title= 'Horizon 1 Error plot:')

for i in metrics_1.columns:
    plot_metric(metrics_1,i, title='Horizon 1:')

# Horizon = 3:

In [ ]:
predsPath_3= Path('../Predictions/Horizon3')
predsDf_3=get_predictions(predsPath_3)

metrics_3= calc_metrics(predsDf_3)
display(metrics_3.sort_values(main_metric, axis=0))

error_plot(predsDf_3,model='all',absolute=False, title= 'Horizon 3 Error plot:')
for i in metrics_3.columns:
    plot_metric(metrics_3,i, title='Horizon 3:')

# Horizon = 6:

In [ ]:
predsPath_6= Path('../Predictions/Horizon6')
predsDf_6=get_predictions(predsPath_6)

metrics_6= calc_metrics(predsDf_6)
display(metrics_6.sort_values(main_metric, axis=0))

error_plot(predsDf_6,model='all',absolute=False, title= 'Horizon 6 Error plot:')
for i in metrics_6.columns:
    plot_metric(metrics_6,i, title='Horizon 6:')

# Horizon = 12:

In [ ]:
metrics_12= calc_metrics(predsDf, horizon=12)
display(metrics_12.sort_values(main_metric, axis=0))

# Analysis:

In [ ]:
from matplotlib import pyplot as plt

metrics_12= calc_metrics(predsDf_12)
display(metrics_12.sort_values(main_metric, axis=0))

for model in predsDf.columns:
    plt.plot(predsDf.index, predsDf[model])

plt.legend(predsDf.columns)
plt.show()

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(10, 5))
ax = plt.axes()
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
plt.locator_params(axis='x', nbins=10)

for model in predsDf.columns:
    plt.plot(predsDf.index[:int(len(predsDf)/4)], predsDf[model][:int(len(predsDf)/4)])

plt.legend(predsDf.columns)
plt.show()